In [ ]:
#Import your data and necessary libraries

import requests
import json
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize

BASE_URL = "https://api.genius.com"
TOKEN = "YOUR_TOKEN"
HEADERS = {"Authorization": f"Bearer {TOKEN}"}

def search_artist(artist_name):
    url = BASE_URL + "/search"
    params = {"q": artist_name}
    response = requests.get(url, params=params, headers=HEADERS)
    return response.json()

def get_song_url(song_id):
    url = BASE_URL + f"/songs/{song_id}"
    response = requests.get(url, headers=HEADERS)
    return response.json()

def extract_lyrics_from_url(url):
    # You will need BeautifulSoup to scrape lyrics because Genius doesn't return them via API
    import requests
    from bs4 import BeautifulSoup

    page = requests.get(url)
    html = BeautifulSoup(page.text, "html.parser")
    lyrics = html.find("div", class_="lyrics") or html.find("div", {"data-lyrics-container": "true"})
    return lyrics.get_text(separator=" ").strip()


In [4]:
#Organize our dataset

song_data = {
    "artist": "",
    "genre": "",
    "title": "",
    "release_year": "",
    "lyrics": "",
    "city": "",
    "gender": "",
}


In [5]:
#Identify nounds, verbs, and adjectives in the lyrics using NLTK
def get_pos_counts(text):
    words = word_tokenize(text.lower())
    tagged = pos_tag(words)

    nouns = [w for w, t in tagged if t.startswith("NN")]
    verbs = [w for w, t in tagged if t.startswith("VB")]
    adjs  = [w for w, t in tagged if t.startswith("JJ")]

    return {
        "nouns": Counter(nouns),
        "verbs": Counter(verbs),
        "adjectives": Counter(adjs)
    }
